# Convert the json to networkx graphs format

In [2]:
import json
PATH = "/local/home/jthomm/GraphCLIP/datasets/visual_genome/raw/"
# load json file
with open(PATH + "realistic_adversarial_attributes_gt_accepted.json") as f:
    gt = json.load(f)
with open(PATH + "realistic_adversarial_attributes_adv_accepted.json") as f:
    adv = json.load(f)

In [3]:
import torch
filtered_graphs_path = "/local/home/jthomm/GraphCLIP/datasets/visual_genome/processed/"
filtered_graphs = filtered_graphs = torch.load(filtered_graphs_path + "filtered_graphs.pt")

In [8]:
from tqdm import tqdm
graph_dict = {}
for g in tqdm(filtered_graphs):
    graph_dict[g.image_id] = g
dataset_graphs = []
for sample in gt:
    image_id = sample["image_id"]
    obj1 = sample["objects"][0]
    obj2 = sample["objects"][1]
    dataset_graphs.append((graph_dict[image_id], obj1, obj2,"gt"))
for sample in adv:
    image_id = sample["image_id"]
    obj1 = sample["objects"][0]
    obj2 = sample["objects"][1]
    dataset_graphs.append((graph_dict[image_id], obj1, obj2,"adv"))

100%|██████████| 97216/97216 [00:00<00:00, 1072840.61it/s]


KeyError: 150323

# Evaluate

In [ ]:
from jt_training import get_dataloader, train_one_epoch, evaluate, get_free_gpu

mode = "text_embeddings"

def load_model(clip_model_type, clip_pretrained_dataset, n_rel_classes, n_obj_classes, n_attr_classes, shallow=True, input_mode="text_embeddings"):
    from open_clip.jt_ViT_RelClassifier_lightning import ViT_RelClassifier
    model = ViT_RelClassifier(n_rel_classes, n_obj_classes, n_attr_classes, clip_model_type, clip_pretrained_dataset, shallow=shallow, mode=input_mode)
    prepocess_function = model.preprocess
    device = get_free_gpu(min_mem=20000)
    print(f"Using device {device}")
    model.to(device)
    return model, prepocess_function, device

In [ ]:
import torch
clip_model_type = 'ViT-B/32' # 'ViT-L-14' #'ViT-B/32'
clip_pretrained_dataset = 'laion400m_e32' # 'laion2b_s32b_b82k' #'laion400m_e32'
image_dir = "/local/home/jthomm/GraphCLIP/datasets/visual_genome/raw/VG/"
metadata_path = "/local/home/jthomm/GraphCLIP/datasets/visual_genome/processed/"

model, prepocess_function, device = load_model(clip_model_type, clip_pretrained_dataset, 100, 200, 100)
loaded = torch.load("/local/home/jthomm/GraphCLIP/experiments/2023-05-05/vision_transformer_12/best_rel_model.ckpt")
# loaded = torch.load("/local/home/jthomm/GraphCLIP/experiments/2023-05-09/vision_transformer_5/best_rel_model.ckpt", map_location=device)
print(loaded.keys())
model.load_state_dict(loaded['state_dict'])


In [ ]:
from jt_training import get_realistic_graphs_dataset_ViT
dataset_orig, dataset_adv, list_to_iterate = get_realistic_graphs_dataset_ViT(prepocess_function, image_dir, mode=mode, version='v2')
print(len(list_to_iterate))

In [ ]:
import random
import PIL
from tqdm import tqdm
dataset_orig, dataset_adv, list_to_iterate = get_realistic_graphs_dataset_ViT(prepocess_function, image_dir, mode=mode, version='v2')
correct = 0
total = 0
for d in tqdm(list_to_iterate):
    original_graph = d['original_graph']
    adv_graph = d['adv_graph']
    changed_edge = d['changed_edge']
    image_id = original_graph.image_id

    image, full_text_clip_embd, rel_label, obj1_label, obj2_label, attr_label, rel_mask, attr_mask = dataset_orig.getitem_from_id_edge(image_id, changed_edge, mode=mode)
    image2, _, rel_label_adv, obj1_label_adv, obj2_label_adv, attr_label_adv, rel_mask_adv, attr_mask_adv = dataset_adv.getitem_from_id_edge(image_id, changed_edge, mode=mode)
    assert (image == image2).all()
    # print("original relationship label:", rel_label)
    # print("adversarial relationship label:", rel_label_adv)
    # print(full_text_clip_embd.shape)
    rel, obj1, obj2, attr = model(image.unsqueeze(0).to(device), full_text_clip_embd.unsqueeze(0).to(device))
    rel_adv, obj1_adv, obj2_adv, attr_adv = model(image2.unsqueeze(0).to(device), full_text_clip_embd.unsqueeze(0).to(device))
    original_rel_confidence = rel[0][rel_label].item()
    adversarial_rel_confidence = rel_adv[0][rel_label_adv].item()
    if original_rel_confidence > adversarial_rel_confidence:
        correct += 1
    else:
        rel_name_dict = dataset_orig.rel_classes # a dict with the relationship names as keys and the ids as values
        rel_name = [key for key, value in rel_name_dict.items() if value == rel_label][0]
        rel_name_adv = [key for key, value in rel_name_dict.items() if value == rel_label_adv][0]
        print("orig rel:", rel_name, "adv rel:", rel_name_adv, "orig conf:", original_rel_confidence, "adv conf:", adversarial_rel_confidence)
    total += 1
    # print("original relationship confidence:", original_rel_confidence)
    # print("adversarial relationship confidence:", adversarial_rel_confidence)
    # break
print("accuracy:", correct/total)